In [50]:
#load all necessary libraries
import pandas as pd 
import numpy as np 
import scipy as scp
import seaborn as sns
import random
import sklearn
#import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

df_test = pd.read_spss('data/newdata/مهندسی.sav')
df_test.head()

,ID,Boomi,Sen,Maghtah,Moadel,Doreh,Meliyat,Vaziyattahol,Farzandan,Sal,Komakmali,Jensiyat,Leave,Remove,Tasmim
0,810100001.0,0.0,29.0,PhD,14.86,Rozaneh,Irari,Mojarad,0.0,1400,Bedon-Komak,Mard,No,No,No
1,810100004.0,Boomi,37.0,PhD,16.04,Shabaneh,Irari,Motahel,1.0,1400,Bedon-Komak,Mard,No,No,No
2,810100005.0,0.0,29.0,PhD,14.51,Rozaneh,Irari,Mojarad,0.0,1400,Bedon-Komak,Zan,No,No,No
3,810100006.0,Boomi,31.0,PhD,17.92,Shabaneh,Irari,Motahel,0.0,1400,Bedon-Komak,Mard,No,No,No
4,810100008.0,Boomi,35.0,PhD,17.51,Rozaneh,Irari,Motahel,2.0,1400,Bedon-Komak,Zan,No,No,No


In [51]:
import pandas as pd

# create a sample data frame
df = pd.DataFrame({
    'Column1': ['A', 'B', 'C', 'A'],
    'Column2': ['X', 'Y', 'X', 'Z'],
    'Column3': ['Yes', 'No', 'Yes', 'Yes']
})
print(df)
# create a dictionary to map unique values to numbers
mapping_dict = {}
for col in df.columns:
    unique_values = df[col].unique()
    mapping_dict[col] = {value: i for i, value in enumerate(unique_values)}

# replace values in data frame with mapped values
df.replace(mapping_dict, inplace=True)

print(df)


  Column1 Column2 Column3
0       A       X     Yes
1       B       Y      No
2       C       X     Yes
3       A       Z     Yes
   Column1  Column2  Column3
0        0        0        0
1        1        1        1
2        2        0        0
3        0        2        0


In [52]:
# create a dictionary to map unique values to numbers
mapping_dict = {}
variables = list(set(df_test.columns) - {"ID", "Sen", "Moadel", "Farzandan", "Sal"})
for col in variables:
    unique_values = df_test[col].unique()
    mapping_dict[col] = {value: i for i, value in enumerate(unique_values)}

# replace values in data frame with mapped values
df_test.replace(mapping_dict, inplace=True)
df_test.head()

,ID,Boomi,Sen,Maghtah,Moadel,Doreh,Meliyat,Vaziyattahol,Farzandan,Sal,Komakmali,Jensiyat,Leave,Remove,Tasmim
0,810100001.0,0,29.0,0,14.86,0,0,0,0.0,1400,0,0,0,0,0
1,810100004.0,1,37.0,0,16.04,1,0,1,1.0,1400,0,0,0,0,0
2,810100005.0,0,29.0,0,14.51,0,0,0,0.0,1400,0,1,0,0,0
3,810100006.0,1,31.0,0,17.92,1,0,1,0.0,1400,0,0,0,0,0
4,810100008.0,1,35.0,0,17.51,0,0,1,2.0,1400,0,1,0,0,0


In [53]:
print(df_test[df_test['Leave'] == 1].shape)
print(df_test[df_test['Remove'] == 1].shape)
#print(df_test[df_test['Tasmim'] == 1].shape)
print(df_test.shape)

(7, 15)
(6, 15)
(3494, 15)


In [54]:
def dummy_fill(df):
    SL = df['Leave']
    CF = df['Remove']

    if SL == 1:
        return 'Leave'
    if CF == 1:
        return 'Remove'
    else: 
        return '0'       

df_test['newClass'] = df_test.apply(dummy_fill, axis=1)

In [55]:
df_test.head()
print("non zero percentage:", 100*len(df_test[(df_test['newClass'] != '0')]) / len(df_test))

non zero percentage: 0.3720663995420721


In [56]:
df_test.dropna(inplace=True)
df_test.isna().sum()

ID              0
Boomi           0
Sen             0
Maghtah         0
Moadel          0
Doreh           0
Meliyat         0
Vaziyattahol    0
Farzandan       0
Sal             0
Komakmali       0
Jensiyat        0
Leave           0
Remove          0
Tasmim          0
newClass        0
dtype: int64

In [57]:
X = df_test.drop(['Leave', 'Remove', 'newClass'], axis=1) 
y = df_test['newClass']
#y=y.astype('object')

print("Columns for training:", list(X.columns.values)) 

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

#print the tunable parameters (They were not tuned in this example, everything kept as default)
params = model1.get_params()
print(params)

#Print model parameters
#print('Intercept: \n', model1.intercept_)
#print('Coefficients: \n', model1.coef_)

#Create a confusion matrix
#y_test as first argument and the preds as second argument 
confusion_matrix(y_test, preds)

#transform confusion matrix into array
#the matrix is stored in a vaiable called confmtrx
confmtrx = np.array(confusion_matrix(y_test, preds))

#Accuracy statistics

print('Accuracy Score:', metrics.accuracy_score(y_test, preds))  

#Create classification report
class_report=classification_report(y_test, preds)
print(class_report)

Columns for training: ['ID', 'Boomi', 'Sen', 'Maghtah', 'Moadel', 'Doreh', 'Meliyat', 'Vaziyattahol', 'Farzandan', 'Sal', 'Komakmali', 'Jensiyat', 'Tasmim']
(2723, 13)
(681, 13)
(2723,)
(681,)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: F

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'none', 'random_state': 0, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Accuracy Score: 0.9955947136563876
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       676
       Leave       0.00      0.00      0.00         2
      Remove       1.00      0.67      0.80         3

    accuracy                           1.00       681
   macro avg       0.67      0.56      0.60       681
weighted avg       0.99      1.00      0.99       681



In [58]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)

print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       676
       Leave       1.00      1.00      1.00         2
      Remove       1.00      1.00      1.00         3

    accuracy                           1.00       681
   macro avg       1.00      1.00      1.00       681
weighted avg       1.00      1.00      1.00       681

[[676   0   0]
 [  0   2   0]
 [  0   0   3]]
